In [28]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import yfinance as yf

In [50]:
load_dotenv()
path_dados_indices = os.getenv('PATH_DADOS_GERAIS')

# Pegar dados do ibovespa
tickerData = yf.Ticker('^BVSP')
# Importar dados usando taxa de amostragem de 1 mês
tickerDf = tickerData.history(period='1mo', start='2010-01-01', end='2024-09-30')
# Deixar apenas data no indíce
tickerDf.index = tickerDf.index
ibov = tickerDf[['Close']]
ibov = ibov.rename_axis('DATE')
ibov.index = ibov.index.tz_localize(None)


fgv_index = pd.read_csv(path_dados_indices + 'fgv_variaveis.csv')
iee_index = pd.read_csv(path_dados_indices + 'iee.csv', sep=';')
incc_index = pd.read_csv(path_dados_indices + 'INCC.csv')
igpm_index = pd.read_csv(path_dados_indices + 'new_igpm.csv', sep=';')
ipca_index = pd.read_csv(path_dados_indices + 'new_ipca.csv', sep=';')
selic_index = pd.read_csv(path_dados_indices + 'new_selic.csv', sep=';')
vix_index = pd.read_csv(path_dados_indices + 'VIX_History.csv')[['DATE', 'CLOSE']]
fgv_index2 = pd.read_csv(path_dados_indices + 'xgvxConsulta.csv', sep=';')

In [51]:
fgv_index2 = pd.read_csv(path_dados_indices + 'xgvxConsulta.csv', sep=';')

In [52]:
#Removendo colunas com poucos valores ou não utilizadas
fgv_index = fgv_index.drop(columns=['Índice de Variação de Aluguéis Residenciais (IVAR) - Média Nacional (1463201)'])
fgv_index2 = fgv_index2.drop(columns=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 12'])

In [53]:
#Ajustando formato da coluna de data para datetime e filtrando janela necessária
fgv_index2['Data'] = pd.to_datetime(fgv_index2['Data'], format='%m/%Y')
fgv_index2 = fgv_index2.rename(columns={'Data': 'data'})
fgv_index2 = fgv_index2[fgv_index2['data'] >= '2010-01-01']
fgv_index2 = fgv_index2[fgv_index2['data'] <= '2024-12-31']

In [54]:
#Tratamento dos dados
fgv_index2.set_index('data', inplace=True)
fgv_index2 = fgv_index2.replace({r'\.': '', ',': '.'}, regex=True)
fgv_index2 = fgv_index2.replace('-',0)
fgv_index2 = fgv_index2.replace(' - ',0)
fgv_index2 = fgv_index2.apply(pd.to_numeric)

In [55]:
#mudar a taxa de amostragem dos dados de mensal para trimestral
fgv_index2 = fgv_index2.resample('QE').mean()
fgv_index2.tail()

,1,2,3,4,5,6,7,8,9
data,,,,,,,,,
2023-12-31,5.220000,4.583333,4.952167,12.236667,6.550000,14892.666667,15977.000000,8694.000000,59.453333
2024-03-31,5.330000,3.093333,4.952367,11.266667,6.530000,15421.000000,15791.000000,9017.000000,60.193333
2024-06-30,5.330000,4.573333,5.216433,10.503333,6.680000,14567.000000,16411.666667,8715.000000,61.130000
2024-09-30,5.263333,4.216667,5.544800,10.400000,6.903333,13777.666667,16986.666667,7932.333333,61.383333
2024-12-31,4.650000,4.500000,5.842133,10.946667,7.430000,15047.333333,16661.333333,8711.000000,61.360000


In [56]:
#renomear as colunas para facilitar o entendimento
fgv_index2 = fgv_index2.rename(columns={"1": "Juros EUA", "2": "Tbonds", "3": "Cotacao dolar", "4": "CDI",
                                         "5": "TJLP", "6": "Energia res", "7": "Energia ind", "8": "Energia comer",
                                         "9": "Divida/PIB"})

In [57]:
fgv_index2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2010-03-31 to 2024-12-31
Freq: QE-DEC
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Juros EUA      60 non-null     float64
 1   Tbonds         60 non-null     float64
 2   Cotacao dolar  60 non-null     float64
 3   CDI            60 non-null     float64
 4   TJLP           60 non-null     float64
 5   Energia res    60 non-null     float64
 6   Energia ind    60 non-null     float64
 7   Energia comer  60 non-null     float64
 8   Divida/PIB     60 non-null     float64
dtypes: float64(9)
memory usage: 4.7 KB


In [58]:
vix_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    8763 non-null   object 
 1   CLOSE   8763 non-null   float64
dtypes: float64(1), object(1)
memory usage: 137.1+ KB


In [59]:
#converter a primeira coluna dos dataframes para datetime
iee_index['Data'] = pd.to_datetime(iee_index['Data'])
incc_index['Data'] = pd.to_datetime(incc_index['Data'])
igpm_index['DATA'] = pd.to_datetime(igpm_index['DATA'])
ipca_index['DATA'] = pd.to_datetime(ipca_index['DATA'])
selic_index['DATA'] = pd.to_datetime(selic_index['DATA'])
vix_index['DATE'] = pd.to_datetime(vix_index['DATE'])

Filtrar a janela de tempo que será utilizada

In [60]:
# Filtrar datas entre 2010 e 2024 para todos os dataframes trimestrais

def filtra_periodo(df):
    col_data = df.columns[0]
    df = df.copy()
    df[col_data] = pd.to_datetime(df[col_data]).dt.tz_localize(None)
    return df[(df[col_data] >= '2010-01-01') & (df[col_data] <= '2024-12-31')]

ibov_trimestral_filtrado = filtra_periodo(ibov)
iee_trimestral_filtrado = filtra_periodo(iee_index)
incc_trimestral_filtrado = filtra_periodo(incc_index)
igpm_trimestral_filtrado = filtra_periodo(igpm_index)
ipca_trimestral_filtrado = filtra_periodo(ipca_index)
selic_trimestral_filtrado = filtra_periodo(selic_index)
vix_trimestral_filtrado = filtra_periodo(vix_index)

# Padronizar as colunas de data para datetime64[ns] sem timezone
for df in [ibov_trimestral_filtrado, iee_trimestral_filtrado, incc_trimestral_filtrado, igpm_trimestral_filtrado, ipca_trimestral_filtrado, selic_trimestral_filtrado, vix_trimestral_filtrado]:
    df[df.columns[0]] = pd.to_datetime(df[df.columns[0]]).dt.tz_localize(None)

In [61]:
fgv_index.tail()

,Data,IPAOG-DI Ipa - todos os itens - Nro. Índice(1477007),INCC-Brasil-DI-Todos os itens(1464783),Indicador de Incerteza da Economia Brasil (IIE-Br)(1428452),Indicador de Incerteza da Economia Brasil - Componente Mídia (Mídia - IIE-Br)(1463188),Indicador de Incerteza da Economia Brasil - Componente Expactativas (Expec IIE-Br)(1463189),Indicador Antecedente de Emprego (IA Emp) com ajuste sazonal(1416205),Indicador Antecedente de Emprego (IA Emp) com ajuste sazonal(1416205) (Variação Percentual em Relação ao Período Anterior),PIB a preços de 1995(1428699),PIB a preços correntes(1428698),...,Índice de volume trimestral com ajuste sazonal - Valor Adicionado(1428701),Índice de volume trimestral com ajuste sazonal - Valor Adicionado - Agropecuária(1428702),Índice de volume trimestral com ajuste sazonal - Valor Adicionado - Indústria(1428703),Índice de volume trimestral com ajuste sazonal - Valor Adicionado - Serviços(1428704),Índice da Taxa de Câmbio Real - (Real / Dólar (EUA))(1003592),Índice da Taxa de Câmbio Real - (Real / Iene (Japão))(1003593),Índice da Taxa de Câmbio Real - (Real / Peso (Argentina))(1003594),Índice da Taxa de Câmbio Real - (Real / Euro (Zona do Euro))(1003595),Índice da Taxa de Câmbio Real - (Real / Libra (Reino Unido))(1003596),Índice da Taxa de Câmbio Efetiva Real - (Real / Cesta de Moedas)(1003597)
55,2023-12-31,1262.023333,1085.846667,109.433333,110.733333,100.366667,75.766667,0.413333,107604.333333,9.551003e+05,...,182.279,255.028,140.986,194.661,66.356667,26.940000,40.240000,62.293333,48.456667,53.960000
56,2024-03-31,1255.267667,1093.224333,105.800000,107.833333,95.200000,78.733333,0.936667,109054.333333,9.177333e+05,...,183.742,269.740,141.673,196.433,67.046667,26.463333,37.846667,56.986667,49.543333,53.236667
57,2024-06-30,1268.807667,1110.367667,110.000000,111.600000,99.400000,79.500000,-0.036667,112034.333333,9.737430e+05,...,186.301,263.528,142.714,199.515,70.283333,27.070000,37.826667,63.150000,51.860000,56.166667
58,2024-09-30,1295.576333,1134.363000,108.633333,107.300000,110.266667,82.133333,0.976667,114685.666667,9.966380e+05,...,187.637,260.698,144.144,201.002,73.223333,29.793333,40.023333,67.623333,55.293333,58.793333
59,2024-12-31,1332.694000,1149.170000,104.800000,104.000000,106.200000,81.700000,0.000000,113745.000000,1.031746e+06,...,NaN,NaN,NaN,NaN,72.350000,29.400000,39.320000,66.290000,54.630000,57.450000


Ajustando a taxa de amostragem dos dados diários para trimestrais

In [62]:
# Ajustando a taxa de amostragem dos dados diários para trimestrais
from pandas.tseries.offsets import QuarterEnd

def amostra_trimestral(df, data_col):
    df = df.copy()
    df[data_col] = pd.to_datetime(df[data_col]).dt.tz_localize(None)
    df.set_index(data_col, inplace=True)
    df_trimestral = df.resample('QE').last().reset_index()
    return df_trimestral


def amostra_trimestral_mean(df, data_col):
    df = df.copy()
    df[data_col] = pd.to_datetime(df[data_col]).dt.tz_localize(None)
    df.set_index(data_col, inplace=True)
    df_trimestral = df.resample('QE').mean().reset_index()
    return df_trimestral

def amostra_trimestral_agregada(df, data_col, valor_col):
    df = df.copy()
    df[data_col] = pd.to_datetime(df[data_col], errors='coerce')
    if hasattr(df[data_col].dt, 'tz') and df[data_col].dt.tz is not None:
        df[data_col] = df[data_col].dt.tz_localize(None)
    df = df.dropna(subset=[data_col, valor_col])
    df.set_index(data_col, inplace=True)
    df = df.sort_index()
    df[valor_col] = df[valor_col].astype(float) / 100
    df_trimestral = (df[valor_col] + 1).resample('QE').prod() - 1
    df_trimestral = df_trimestral.reset_index()
    return df_trimestral

# Exemplo para cada dataframe:
ibov_index_trimestral = ibov.resample('QE').last().reset_index()
iee_index_trimestral = amostra_trimestral(iee_trimestral_filtrado, data_col=iee_trimestral_filtrado.columns[0])
selic_index_trimestral = amostra_trimestral_mean(selic_trimestral_filtrado, data_col=selic_trimestral_filtrado.columns[0])
vix_index_trimestral = amostra_trimestral(vix_index, data_col=vix_index.columns[0])

# Para índices percentuais:
igpm_index_trimestral = amostra_trimestral_agregada(igpm_trimestral_filtrado, data_col=igpm_trimestral_filtrado.columns[0], valor_col=igpm_trimestral_filtrado.columns[1])
ipca_index_trimestral = amostra_trimestral_agregada(ipca_trimestral_filtrado, data_col=ipca_trimestral_filtrado.columns[0], valor_col=ipca_trimestral_filtrado.columns[1])
incc_index_trimestral = amostra_trimestral_agregada(incc_trimestral_filtrado, data_col=incc_trimestral_filtrado.columns[0], valor_col=incc_trimestral_filtrado.columns[1])

In [63]:
iee_index_trimestral.columns = ['data', 'iee']
selic_index_trimestral.columns = ['data', 'selic']
incc_index_trimestral.columns = ['data', 'incc']
ipca_index_trimestral.columns = ['data', 'ipca']
igpm_index_trimestral.columns = ['data', 'igpm']
vix_index_trimestral.columns = ['data', 'vix']
ibov_index_trimestral.columns = ['data', 'ibov']
fgv_index['Data'] = pd.to_datetime(fgv_index['Data'])
fgv_index = fgv_index.rename(columns={'Data': 'data'})

In [64]:
ibov_index_trimestral.tail()

,data,ibov
54,2023-09-30,116565.0
55,2023-12-31,134185.0
56,2024-03-31,128106.0
57,2024-06-30,123907.0
58,2024-09-30,132730.0


In [65]:
#Juntar os dataframes ibov_index_trimestral  iee_index_trimestral   selic_index_trimestral  vix_index_trimestral   igpm_index_trimestral ipca_index_trimestral incc_index_trimestral
df_junto = ibov_index_trimestral.merge(iee_index_trimestral, on='data', )
df_junto = df_junto.merge(selic_index_trimestral, on='data', suffixes=('', '_selic'))
df_junto = df_junto.merge(vix_index_trimestral, on='data', suffixes=('', '_vix'))
df_junto = df_junto.merge(igpm_index_trimestral, on='data', suffixes=('', '_igpm'))
df_junto = df_junto.merge(ipca_index_trimestral, on='data', suffixes=('', '_ipca'))
df_junto = df_junto.merge(incc_index_trimestral, on='data', suffixes=('', '_incc'))
df_junto = df_junto.merge(fgv_index, on='data', suffixes=('', '_fgv'))
df_junto = df_junto.merge(fgv_index2, on='data', suffixes=('', '_fgv2'))

In [66]:
len(df_junto.columns)

48

In [67]:
df_junto.tail()

,data,ibov,iee,selic,vix,igpm,ipca,incc,IPAOG-DI Ipa - todos os itens - Nro. Índice(1477007),INCC-Brasil-DI-Todos os itens(1464783),...,Índice da Taxa de Câmbio Efetiva Real - (Real / Cesta de Moedas)(1003597),Juros EUA,Tbonds,Cotacao dolar,CDI,TJLP,Energia res,Energia ind,Energia comer,Divida/PIB
54,2023-09-30,116565.0,"85.443,10",13.216667,17.52,-0.004922,0.006112,0.006111,1241.658000,1079.047333,...,53.983333,5.150000,4.236667,4.879833,13.277367,7.000000,13088.333333,15922.000000,7662.333333,58.630000
55,2023-12-31,134185.0,"94.956,92",12.250000,12.45,0.018410,0.010836,0.002701,1262.023333,1085.846667,...,53.960000,5.220000,4.583333,4.952167,12.236667,6.550000,14892.666667,15977.000000,8694.000000,59.453333
56,2024-03-31,128106.0,"88.654,31",11.250000,13.01,-0.009182,0.014155,0.006815,1255.267667,1093.224333,...,53.236667,5.330000,3.093333,4.952367,11.266667,6.530000,15421.000000,15791.000000,9017.000000,60.193333
57,2024-06-30,123907.0,"88.289,36",10.583333,12.44,0.020225,0.010535,0.021043,1268.807667,1110.367667,...,56.166667,5.330000,4.573333,5.216433,10.503333,6.680000,14567.000000,16411.666667,8715.000000,61.130000
58,2024-09-30,132730.0,"92.892,10",10.500000,16.73,0.015274,0.008015,0.019526,1295.576333,1134.363000,...,58.793333,5.263333,4.216667,5.544800,10.400000,6.903333,13777.666667,16986.666667,7932.333333,61.383333


In [71]:
#converter coluna iee para float
df_junto['iee'] = df_junto['iee'].replace({r'\.': '', ',': '.'}, regex=True)
df_junto['iee'] = df_junto['iee'].astype(float)

In [72]:
path_dados_tratados = os.getenv('PATH_DADOS_TRATADOS')
df_junto.to_csv(path_dados_tratados + "dados_indices.csv", sep=';')